# ID Card Segmentation

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras import backend as K
import json
import os
import random
import re
import shutil
import wget
import zipfile
from PIL import Image
from glob import glob

## Utility Functions

In [ ]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

In [ ]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped


def findLargestCountours(cntList, cntWidths):
    newCntList = []
    newCntWidths = []

    first_largest_cnt_pos = cntWidths.index(max(cntWidths))

    newCntList.append(cntList[first_largest_cnt_pos])
    newCntWidths.append(cntWidths[first_largest_cnt_pos])

    cntList.pop(first_largest_cnt_pos)
    cntWidths.pop(first_largest_cnt_pos)

    seccond_largest_cnt_pos = cntWidths.index(max(cntWidths))

    newCntList.append(cntList[seccond_largest_cnt_pos])
    newCntWidths.append(cntWidths[seccond_largest_cnt_pos])

    cntList.pop(seccond_largest_cnt_pos)
    cntWidths.pop(seccond_largest_cnt_pos)
    return newCntList, newCntWidths

In [ ]:
def convert_object(mask, image):
    gray = mask
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    gray = cv2.medianBlur(gray, 5)
    edged = cv2.Canny(np.uint8(gray), 30, 400)
    countours, _ = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    cnts = sorted(countours, key=cv2.contourArea, reverse=True)
    screenCntList = []
    scrWidths = []
    for cnt in cnts:
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
        screenCnt = approx

        if (len(screenCnt) == 4):
            (X, Y, W, H) = cv2.boundingRect(cnt)
            screenCntList.append(screenCnt)
            scrWidths.append(W)

    if len(scrWidths) != 2:
        print('ID Card not found.')
        return None
    else:
        screenCntList, scrWidths = findLargestCountours(screenCntList, scrWidths)

        if not len(screenCntList) >= 2:  # there is no rectangle found
            return None
        elif scrWidths[0] != scrWidths[1]:  # mismatch in rect
            return None

        pts = screenCntList[0].reshape(4, 2)
        warped = four_point_transform(image, pts)
        return warped

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true), -1) + K.sum(K.square(y_pred), -1) + smooth)

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)


In [ ]:
def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def iou_loss(true, pred):
    intersection = true * pred
    notTrue = 1 - true
    union = true + (notTrue * pred)
    return K.sum(intersection)/K.sum(union)

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))

    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.

    Ref: https://en.wikipedia.org/wiki/Jaccard_index

    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

## Prepare Dataset

In [ ]:
download_links = [
    'ftp://smartengines.com/midv-500/dataset/12_deu_drvlic_new.zip',
    'ftp://smartengines.com/midv-500/dataset/13_deu_drvlic_old.zip',
    'ftp://smartengines.com/midv-500/dataset/14_deu_id_new.zip',
    'ftp://smartengines.com/midv-500/dataset/15_deu_id_old.zip',
    'ftp://smartengines.com/midv-500/dataset/16_deu_passport_new.zip',
    'ftp://smartengines.com/midv-500/dataset/17_deu_passport_old.zip']

'''
download_links = ['ftp://smartengines.com/midv-500/dataset/01_alb_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/02_aut_drvlic_new.zip',
                  'ftp://smartengines.com/midv-500/dataset/03_aut_id_old.zip',
                  'ftp://smartengines.com/midv-500/dataset/04_aut_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/05_aze_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/06_bra_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/07_chl_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/08_chn_homereturn.zip',
                  'ftp://smartengines.com/midv-500/dataset/09_chn_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/10_cze_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/11_cze_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/12_deu_drvlic_new.zip',
                  'ftp://smartengines.com/midv-500/dataset/13_deu_drvlic_old.zip',
                  'ftp://smartengines.com/midv-500/dataset/14_deu_id_new.zip',
                  'ftp://smartengines.com/midv-500/dataset/15_deu_id_old.zip',
                  'ftp://smartengines.com/midv-500/dataset/16_deu_passport_new.zip',
                  'ftp://smartengines.com/midv-500/dataset/17_deu_passport_old.zip',
                  'ftp://smartengines.com/midv-500/dataset/18_dza_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/19_esp_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/20_esp_id_new.zip',
                  'ftp://smartengines.com/midv-500/dataset/21_esp_id_old.zip',
                  'ftp://smartengines.com/midv-500/dataset/22_est_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/23_fin_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/24_fin_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/25_grc_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/26_hrv_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/27_hrv_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/28_hun_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/29_irn_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/30_ita_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/31_jpn_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/32_lva_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/33_mac_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/34_mda_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/35_nor_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/36_pol_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/37_prt_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/38_rou_drvlic.zip',
                  'ftp://smartengines.com/midv-500/dataset/39_rus_internalpassport.zip',
                  'ftp://smartengines.com/midv-500/dataset/40_srb_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/41_srb_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/42_svk_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/43_tur_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/44_ukr_id.zip',
                  'ftp://smartengines.com/midv-500/dataset/45_ukr_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/46_ury_passport.zip',
                  'ftp://smartengines.com/midv-500/dataset/47_usa_bordercrossing.zip',
                  'ftp://smartengines.com/midv-500/dataset/48_usa_passportcard.zip',
                  'ftp://smartengines.com/midv-500/dataset/49_usa_ssn82.zip',
                  'ftp://smartengines.com/midv-500/dataset/50_xpo_id.zip']
'''

PATH_OFFSET = 40
TARGET_PATH = 'dataset/data/'

TEMP_PATH = 'dataset/temp/'
TEMP_IMAGE_PATH = TEMP_PATH + 'image/'
TEMP_MASK_PATH = TEMP_PATH + 'mask/'

DATA_PATH = 'dataset/train/'

SEED = 230


def read_image(img, label):
    image = cv2.imread(img)
    mask = np.zeros(image.shape, dtype=np.uint8)
    quad = json.load(open(label, 'r'))
    coords = np.array(quad['quad'], dtype=np.int32)
    cv2.fillPoly(mask, coords.reshape(-1, 4, 2), color=(255, 255, 255))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = cv2.resize(mask, (mask.shape[1] // 2, mask.shape[0] // 2))
    image = cv2.resize(image, (image.shape[1] // 2, image.shape[0] // 2))
    mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)[1]
    return image, mask


def download_and_unzip():
    if os.path.exists(TEMP_PATH):
        shutil.rmtree(TEMP_PATH, ignore_errors=True)

    os.mkdir(TEMP_PATH)
    os.mkdir(TEMP_IMAGE_PATH)
    os.mkdir(TEMP_MASK_PATH)

    file_idx = 1

    for link in download_links:
        filename = link[PATH_OFFSET:]
        full_filename = TARGET_PATH + filename
        directory_name = TARGET_PATH + link[PATH_OFFSET:-4]

        print('Collect and prepare datasets...')

        print('Dataset available... ', directory_name)
        if not os.path.exists(directory_name):
            if not os.path.isfile(full_filename):
                print ('Downloading:', link)
                wget.download(link, TARGET_PATH)

            with zipfile.ZipFile(full_filename, 'r') as zip_ref:
                zip_ref.extractall(TARGET_PATH)

        print('Prepare dataset... ', directory_name)
        img_dir_path = './' + directory_name + '/images/'
        gt_dir_path = './' + directory_name + '/ground_truth/'

        if os.path.isfile(img_dir_path + filename + '.tif'):
            os.remove(img_dir_path + filename.replace('.zip', '.tif'))
        if os.path.isfile(gt_dir_path + filename + '.json'):
            os.remove(gt_dir_path + filename.replace('.zip', '.json'))

        for images, ground_truth in zip(sorted(os.listdir(img_dir_path)), sorted(os.listdir(gt_dir_path))):
            img_list = sorted(glob(img_dir_path + images + '/*.tif'))
            label_list = sorted(glob(gt_dir_path + ground_truth + '/*.json'))
            for img, label in zip(img_list, label_list):
                image, mask = read_image(img, label)
                cv2.imwrite(TEMP_IMAGE_PATH + 'image' + str(file_idx) + '.png', image)
                cv2.imwrite(TEMP_MASK_PATH + 'image' + str(file_idx) + '.png', mask)

                file_idx += 1

        print('----------------------------------------------------------------------')


def train_validation_split():
    if os.path.exists(DATA_PATH):
        shutil.rmtree(DATA_PATH, ignore_errors=True)

    folders = ['train_frames/image', 'train_masks/image', 'val_frames/image', 'val_masks/image', 'test_frames/image',
               'test_masks/image']

    for folder in folders:
        os.makedirs(DATA_PATH + folder)

    all_frames = os.listdir(TEMP_IMAGE_PATH)
    all_masks = os.listdir(TEMP_MASK_PATH)

    all_frames.sort(key=lambda var: [int(x) if x.isdigit() else x
                                     for x in re.findall(r'[^0-9]|[0-9]+', var)])
    all_masks.sort(key=lambda var: [int(x) if x.isdigit() else x
                                    for x in re.findall(r'[^0-9]|[0-9]+', var)])

    random.seed(SEED)
    random.shuffle(all_frames)

    train_split = int(0.7 * len(all_frames))
    val_split = int(0.9 * len(all_frames))

    train_frames = all_frames[:train_split]
    val_frames = all_frames[train_split:val_split]
    test_frames = all_frames[val_split:]

    train_masks = [f for f in all_masks if f in train_frames]
    val_masks = [f for f in all_masks if f in val_frames]
    test_masks = [f for f in all_masks if f in test_frames]

    def add_frames(dir_name, image):
        img = Image.open(TEMP_IMAGE_PATH + image)
        img.save(DATA_PATH + '/{}'.format(dir_name) + '/' + image)

    def add_masks(dir_name, image):
        img = Image.open(TEMP_MASK_PATH + image)
        img.save(DATA_PATH + '/{}'.format(dir_name) + '/' + image)

    frame_folders = [(train_frames, 'train_frames/image'), (val_frames, 'val_frames/image'),
                     (test_frames, 'test_frames/image')]
    mask_folders = [(train_masks, 'train_masks/image'), (val_masks, 'val_masks/image'),
                    (test_masks, 'test_masks/image')]

    print('Split images into train, test and validation...')

    for folder in frame_folders:
        array = folder[0]
        name = [folder[1]] * len(array)
        list(map(add_frames, name, array))

    for folder in mask_folders:
        array = folder[0]
        name = [folder[1]] * len(array)
        list(map(add_masks, name, array))


download_and_unzip()
train_validation_split()

## Models

In [ ]:
from keras.layers import *
from keras.models import *
from keras.optimizers import *

def UNET(input_size=(256, 256, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    model.summary()

    return model

## Training Script

In [ ]:
import numpy as np
import os
import random as rn
import tensorflow as tf
import time
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

import models
from utils import metrics

In [ ]:
NO_OF_TRAINING_IMAGES = len(os.listdir('dataset/train/train_frames/image'))
NO_OF_VAL_IMAGES = len(os.listdir('dataset/train/val_frames/image'))

NO_OF_EPOCHS = 500
BATCH_SIZE = 8

IMAGE_SIZE = (256, 256)

SEED = 230
rn.seed(SEED)
np.random.seed(SEED)
tf.set_random_seed(SEED)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
train_image_generator = train_datagen.flow_from_directory('./dataset/train/train_frames',
                                                          target_size=IMAGE_SIZE,
                                                          class_mode=None,
                                                          batch_size=BATCH_SIZE,
                                                          color_mode='grayscale',
                                                          seed=SEED)

train_mask_generator = train_datagen.flow_from_directory('dataset/train/train_masks',
                                                         target_size=IMAGE_SIZE,
                                                         class_mode=None,
                                                         batch_size=BATCH_SIZE,
                                                         color_mode='grayscale',
                                                         seed=SEED)

val_datagen = ImageDataGenerator(rescale=1. / 255)
val_image_generator = val_datagen.flow_from_directory('dataset/train/val_frames',
                                                      target_size=IMAGE_SIZE,
                                                      class_mode=None,
                                                      batch_size=BATCH_SIZE,
                                                      color_mode='grayscale',
                                                      seed=SEED)

val_mask_generator = val_datagen.flow_from_directory('dataset/train/val_masks',
                                                     target_size=IMAGE_SIZE,
                                                     class_mode=None,
                                                     batch_size=BATCH_SIZE,
                                                     color_mode='grayscale',
                                                     seed=SEED)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

model = models.UNET(input_size=(256, 256, 1))

model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy', metrics.mean_iou])

checkpoint = ModelCheckpoint("model.h5", verbose=1, save_best_only=True, save_weights_only=False,
                             monitor='val_mean_iou', mode='max')
earlystopping = EarlyStopping(patience=10, verbose=1, monitor='val_mean_iou', mode='max')
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=3, verbose=1, min_delta=0.000001,
                              monitor='val_mean_iou', mode='max')
tensorboard = TensorBoard(log_dir='./logs/' + time.strftime("%Y%m%d_%H%M%S"), histogram_freq=0,
                          write_graph=True, write_images=True)

model.fit_generator(train_generator, epochs=NO_OF_EPOCHS,
                    steps_per_epoch=(NO_OF_TRAINING_IMAGES // BATCH_SIZE),
                    validation_data=val_generator,
                    validation_steps=(NO_OF_VAL_IMAGES // BATCH_SIZE),
                    callbacks=[checkpoint, earlystopping, reduce_lr, tensorboard])


# Test Predictions

In [ ]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage import img_as_ubyte
from matplotlib import gridspec
from tensorflow.keras.models import load_model

from utils import image
from utils import metrics

IMPORT_FILES = "./test/*.png"
MODEL_FILE = "model.h5"

In [ ]:
def load_data(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = img / 255.0
    height, width = img.shape[:2]
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    img = img.reshape(1, 256, 256, 1)
    return img, height, width

In [ ]:
images_dict = []
for file_path in glob.glob(IMPORT_FILES):
    raw = cv2.imread(file_path)
    raw = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
    img, h, w = load_data(file_path)
    images_dict.append({"raw": raw, "data": img, "height": h, "width": w}) 

In [ ]:
gs = gridspec.GridSpec(1, len(images_dict))

In [ ]:
fig = plt.figure(figsize=(16, 6))
for n in range(len(images_dict)):
    ax = fig.add_subplot(gs[n])
    ax.imshow(images_dict[n]["raw"])

In [ ]:
model = load_model(MODEL_FILE, custom_objects={'mean_iou': metrics.mean_iou})

In [ ]:
for n in range(len(images_dict)):
    data = images_dict[n]    
    data["prediction"] = model.predict(data["data"])[0]
    img = cv2.resize(data["prediction"], (data["width"], data["height"])) 
    img = img_as_ubyte(img)    
    data["mask"] = cv2.threshold(np.array(img), 200, 255, cv2.THRESH_BINARY)[1]

In [ ]:
fig = plt.figure(figsize=(16, 6))
for n in range(len(images_dict)):
    ax = fig.add_subplot(gs[n])
    data = images_dict[n]
    ax.imshow(data["mask"], cmap="gray")

## RESULTS

In [ ]:
fig = plt.figure(figsize=(16, 6))
for n in range(len(images_dict)):
    ax = fig.add_subplot(gs[n])
    data = images_dict[n]    
    res = image.convert_object(data["mask"], data["raw"])
    if res is None:
        res = np.ones((1,1,3), np.uint8)
    ax.imshow(res)